In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier

In [1]:
data = pd.read_csv('diabetic_data.csv')

NameError: name 'pd' is not defined

In [ ]:
cleaning = cleaning.drop_duplicates("patient_nbr")
cleaning = cleaning.drop(['encounter_id','patient_nbr','examide','citoglipton','weight','payer_code','glimepiride-pioglitazone'], axis=1)

In [ ]:
cleaning.replace('?', np.nan, inplace=True)

In [ ]:
# impute race,diag123 with mode
cleaning.race.fillna(cleaning.race.mode().iloc[0], inplace=True)
cleaning.diag_1.fillna(cleaning.diag_1.mode().iloc[0], inplace=True)
cleaning.diag_2.fillna(cleaning.diag_2.mode().iloc[0], inplace=True)
cleaning.diag_3.fillna(cleaning.diag_3.mode().iloc[0], inplace=True)
cleaning.medical_specialty.fillna(cleaning.medical_specialty.mode().iloc[0], inplace=True)

In [ ]:
med_top_14 = ['InternalMedicine','Emergency/Trauma','Family/GeneralPractice','Cardiology',\
              'Surgery-General','Nephrology','Orthopedics','Orthopedics-Reconstructive','Radiologist','Pulmonology',\
              'Psychiatry','Urology','ObstetricsandGynecology','Surgery-Cardiovascular/Thoracic']

In [ ]:
cleaning['med_spec'] = cleaning['medical_specialty'].copy()

# replace all specialties not in top 14 with 'Other' category
cleaning.loc[~cleaning.med_spec.isin(med_top_14),'med_spec'] = 'Other_Disease'
cleaning.drop(['medical_specialty'], axis=1, inplace=True)

In [ ]:
le_gender = LabelEncoder()
cleaning.gender = le_gender.fit_transform(cleaning.gender)

In [ ]:
check = cleaning
final = pd.get_dummies(check['race'], drop_first=True)
cleaning.drop(['race'],axis=1,inplace=True)
cleaning = pd.concat([final,cleaning], axis=1)

In [ ]:
cleaning.age = le_gender.fit_transform(cleaning.age)
cleaning.max_glu_serum = le_gender.fit_transform(cleaning.max_glu_serum)
cleaning['diabetesMed'] = le_gender.fit_transform(cleaning['diabetesMed'])
cleaning['change'] = le_gender.fit_transform(cleaning['change'])
cleaning['metformin-pioglitazone'] = le_gender.fit_transform(cleaning['metformin-pioglitazone'])
cleaning['metformin-rosiglitazone'] = le_gender.fit_transform(cleaning['metformin-rosiglitazone'])
cleaning['glipizide-metformin'] = le_gender.fit_transform(cleaning['glipizide-metformin'])
cleaning['tolazamide'] = le_gender.fit_transform(cleaning['tolazamide'])
cleaning['tolbutamide'] = le_gender.fit_transform(cleaning['tolbutamide'])
cleaning['acetohexamide'] = le_gender.fit_transform(cleaning['tolbutamide'])
cleaning['insulin'] = le_gender.fit_transform(cleaning['insulin'])
cleaning['max_glu_serum'] = le_gender.fit_transform(cleaning['max_glu_serum'])
cleaning['glyburide-metformin'] = le_gender.fit_transform(cleaning['glyburide-metformin'])
cleaning['miglitol'] = le_gender.fit_transform(cleaning['miglitol'])
cleaning['rosiglitazone'] = le_gender.fit_transform(cleaning['rosiglitazone'])
cleaning['glyburide'] = le_gender.fit_transform(cleaning['glyburide'])
cleaning['glipizide'] = le_gender.fit_transform(cleaning['glipizide'])
cleaning['glimepiride'] = le_gender.fit_transform(cleaning['glimepiride'])
cleaning['chlorpropamide'] = le_gender.fit_transform(cleaning['chlorpropamide'])
cleaning['nateglinide'] = le_gender.fit_transform(cleaning['nateglinide'])
cleaning['repaglinide'] = le_gender.fit_transform(cleaning['repaglinide'])
cleaning['metformin'] = le_gender.fit_transform(cleaning['metformin'])
cleaning['acarbose'] = le_gender.fit_transform(cleaning['acarbose'])
cleaning['A1Cresult'] = le_gender.fit_transform(cleaning['A1Cresult'])
cleaning['pioglitazone'] = le_gender.fit_transform(cleaning['pioglitazone'])
cleaning['troglitazone'] = le_gender.fit_transform(cleaning['troglitazone'])

In [ ]:
# convert diag_1 with e or v values to other
cleaning.loc[cleaning['diag_1'].str.contains('E',na=False), 'diag_1'] = 0
cleaning.loc[cleaning['diag_1'].str.contains('V',na=False), 'diag_1'] = 0
cleaning.loc[cleaning['diag_2'].str.contains('E',na=False), 'diag_2'] = 0
cleaning.loc[cleaning['diag_2'].str.contains('V',na=False), 'diag_2'] = 0
cleaning.loc[cleaning['diag_3'].str.contains('E',na=False), 'diag_3'] = 0
cleaning.loc[cleaning['diag_3'].str.contains('V',na=False), 'diag_3'] = 0

In [ ]:
# cleaning.diag_1.dtypes
cleaning.diag_1.astype('float')
cleaning.diag_1 = cleaning.diag_1.str.split('.').str[0]
cleaning.diag_1 = cleaning.diag_1.astype('float')

In [ ]:
cleaning['diag_1'].value_counts()
for index, col in cleaning.iterrows():
    if (col['diag_1'] >= 390 and col['diag_1'] < 459  or col['diag_1'] == 785):
        cleaning.loc[index, 'diag_1'] = 'Circulatory'
    elif (col['diag_1'] >= 460 and col['diag_1'] < 519  or col['diag_1'] == 786):
        cleaning.loc[index, 'diag_1'] = 'Respiratory'
    elif (col['diag_1'] >= 520 and col['diag_1'] < 579  or col['diag_1'] == 787):
        cleaning.loc[index, 'diag_1'] = 'Respiratory'
    elif (col['diag_1'] == 250):
        cleaning.loc[index, 'diag_1'] = 'Diabetes'
    elif (col['diag_1'] >= 800 and col['diag_1'] < 999):
        cleaning.loc[index, 'diag_1'] = 'Injury'
    elif (col['diag_1'] >= 710 and col['diag_1'] < 739):
        cleaning.loc[index, 'diag_1'] = 'Musculoskeletal'
    elif (col['diag_1'] >= 580 and col['diag_1'] < 629  or col['diag_1'] == 788):
        cleaning.loc[index, 'diag_1'] = 'Genitourinary'
    elif (col['diag_1'] >= 140 and col['diag_1'] < 239):
        cleaning.loc[index, 'diag_1'] = 'Neoplasms'
    else:
        cleaning.loc[index, 'diag_1'] = 'Other_disease'

In [ ]:
# drop secondary and other diag and focus on primary diag thus diag_1 for now
cleaning.drop(['diag_2','diag_3'],axis=1,inplace=True)

In [ ]:
remove_diag2_3 = pd.get_dummies(cleaning['diag_1'], drop_first=True)
med_spec = pd.get_dummies(cleaning['med_spec'], drop_first=True)
# remove diag_1 after ohe
cleaning.drop(['diag_1'],axis=1,inplace=True)
cleaning.drop(['med_spec'],axis=1,inplace=True)
# concat ohe diag_1 to the main dataset
cleaning = pd.concat([cleaning,remove_diag2_3,med_spec], axis=1)
cleaning.columns

In [ ]:
data = cleaning
data["readmitted"] = np.where(data["readmitted"] == ">30", 1, 0)

In [ ]:
data1 = data.groupby('readmitted').apply(lambda x: x.sample(22240))

In [ ]:
features = data.drop("readmitted", axis = 1)
labels = data["readmitted"]

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state =42)

In [ ]:
features1 = data1.drop("readmitted", axis = 1)
labels1 = data1["readmitted"]

train_features1, test_features1, train_labels1, test_labels1 = train_test_split(features1, labels1, test_size = 0.25)

In [ ]:
rfc = BalancedRandomForestClassifier()
param_grid = { 
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [20, 40, 60, 80, 100],
    'criterion' :['gini', 'entropy']
}

grid_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, scoring='recall', cv= 5)
grid_rfc.fit(train_features, train_labels)
print(grid_rfc.best_params_)
print(grid_rfc.best_score_)
#print(cross_val_score(rfc, features, labels, cv=5))
#rfc.fit(train_features, train_labels)
#rfc.score(test_features, test_labels)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=200)
#print(cross_val_score(clf, features, labels, cv=5))
clf.fit(train_features, train_labels)
#clf.score(test_features, test_labels)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
print(cross_val_score(neigh, features, labels, cv=5, scoring="recall"))
#neigh.fit(train_features, train_labels)
#neigh.score(test_features, test_labels)
neigh.fit(train_features1, train_labels1)
print(neigh.score(test_features1, test_labels1))
predn = neigh.predict(test_features1)
print(classification_report(test_labels1, predn, target_names=["no", "read"]))
print(confusion_matrix(test_labels1, predn))

In [ ]:
#rfc = RandomForestClassifier(class_weight = "balanced", oob_score="True", n_estimators=200, max_features= 'sqrt', max_depth =20)
rfc = BalancedRandomForestClassifier(oob_score="True", n_estimators=300, max_features= 'auto', max_depth =20, criterion='entropy', random_state=42)
rfc.fit(train_features, train_labels)
print(rfc.score(test_features, test_labels))
pred = rfc.predict(test_features)
print(classification_report(test_labels, pred, target_names=["no", "read"]))
print(confusion_matrix(test_labels, pred))

In [ ]:
rfc = RandomForestClassifier(class_weight = "balanced", oob_score="True", n_estimators=200, max_features= 'sqrt', max_depth =80, random_state=42)
rfc.fit(train_features, train_labels)
print(rfc.score(test_features, test_labels))
pred = rfc.predict(test_features)
print(classification_report(test_labels, pred, target_names=["no", "read"]))
print(confusion_matrix(test_labels, pred))

In [ ]:
f = []
for num, x in enumerate(rfc.feature_importances_):
    f.append([features.columns[num], x])
    
temp = sorted(f, key=lambda x: x[1])
temp.reverse()
for x in temp:
    print(x)
